# Example: PATE-CTGAN and tabular data

**IMPORTANT:** refer to the [README]("https://github.com/kasra-hosseini/privgem#credits") for credits.

In [ ]:
# solve issue with autocomplete
%config Completer.use_jedi = False

%load_ext autoreload
%autoreload 2
%matplotlib inline

from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

In [ ]:
import os
import pandas as pd
import sys

## Load a tabular data

In [ ]:
from ctgan import load_demo

data = load_demo()
data.head()

### Name of the columns that are discrete

In [ ]:
discrete_columns = [
    'workclass',
    'education',
    'marital-status',
    'occupation',
    'relationship',
    'race',
    'sex',
    'native-country',
    'income']

## Synthesize using PATE-CTGAN

### Split the data into train/test

In [ ]:
from privgem import tabular_utils

path_save="./test_patectgan/orig_data/orig_data.csv"
path_train="./test_patectgan/orig_data/orig_train.csv"
path_test="./test_patectgan/orig_data/orig_test.csv"

tabular_utils.split_save_orig_data(data,
                                   path_save=path_save,
                                   path_train=path_train,
                                   path_test=path_test,
                                   label_col="income",
                                   test_size=0.25,
                                   random_state=42)

### Instantiate a tabular_patectgan object

In [ ]:
from privgem import tabular_patectgan

# inputs
epsilon = 2
batch_size = 64
noise_multiplier = 0.002
moments_order = 1000
output_save_path = "./test_patectgan/patectgan_training.csv"
device = "default" # or "default" or "cpu" or "cuda:1"

pate_model = tabular_patectgan(verbose=True, 
                               epsilon=epsilon, 
                               batch_size=batch_size, 
                               noise_multiplier=noise_multiplier, 
                               moments_order=moments_order, 
                               output_save_path=output_save_path,
                               device=device)

### train a new model

Note that this can take a long time (depends on the hyperparams)

In [ ]:
pate_model.train(data, discrete_columns)

### Plot the training log file

In [ ]:
from privgem import tabular_utils

tabular_utils.plot_log_patectgan(filename="./test_patectgan/patectgan_training.csv")

### sample and save the output

In [ ]:
synth_output = pate_model.sample(len(data))

path2synth_file = "./test_patectgan/patectgan_001/synthetic_output.csv"
os.makedirs(os.path.dirname(path2synth_file), exist_ok=True)
synth_output.to_csv(path2synth_file, index=False)

synth_output